In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pytesseract
import copy

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [2]:
url = '/Users/yunsu/desktop/python_ocr/Target_test.png'
url2 = '/Users/yunsu/desktop/python_ocr/boseok.png'

In [5]:
print(pytesseract.image_to_string(Image.open(url2)))

Form w-z Wage and Tax Statement
Copy c for EMPLOYEE'S RECORDS

2014

Department of the Treasury-Internal Revenue Service

This information is being furnished to the Internal Revenue Service, If you are required to ﬁle a tax return, a negligence penalty or other sanction may be
imposed on you if this income is taxable and you fail to report it.

b Employer identiﬁcation number

1 3-5565207

 

a Employee's social security number
308-27-4383 OMB No, 1545-0008
1 Wages, tips, other compensation 2 Federal income tax withheld
76579.31 14438.62

 

c Employer's name, address, and ZIP code

KPMG LLP #2381212
717 N. HanNood Street Suite 3100
Dallas, TX 75201-6585

3 Social security wages

4 Social security tax withheld

 

76579.31 4747.92
5 Medicare wages and tips 6 Medicare tax withheld
76579.31 1110.40

 

 

d Control number

 

7 Social security tips

10 Dependent care beneﬁts

3 Allocated tips

 

 

 

 

 

 

 

 

 

 

 

e Employee's ﬁrst name and initial Last name Suff, 11 Nonquali

In [6]:
def make_crack(ful_img, st, end):
    img_list = []

    for i in range(len(st)):
        tmp = ful_img.crop((st[i][1], st[i][0], end[i][1], end[i][0]))
        img_list.append(tmp)
    return img_list


In [23]:
def second(x):
    return x[1]

def make_crack(ful_img, st, end, line_wit):
    img_list = []

    for i in range(len(st)):
        tmp = ful_img.crop((st[i][1]+ line_wit, st[i][0] + line_wit, end[i][1], end[i][0]))
        img_list.append(tmp)
    return img_list


def make_puzzle2(url):
    # 이미지 읽어오기
    src = cv2.imread(url)
    cv2.imshow('src', src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    #이미지 그레이스케일로 변환.
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    # 그레이스케일 이미지를 흑백 반전시키고, Adaptive thresh mean 을 적용해서 잡음 제거해준다.
    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    cv2.imshow('bw', bw)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 수직, 수평 직선을 확인하는 기능. 코드 리팩토링 필요.
    horizontal = bw.copy()
    vertical = bw.copy()
    v_size = vertical.shape[0]//20
    v_size2 = max(1, vertical.shape[0]//100)
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size2))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    v_final = cv2.dilate(v_final, v_struct2, iterations=1)
    
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # h_final은 이미지에서 테이블 수직 선만 표시하는 배열이다.
    h_size = horizontal.shape[1]//20
    # h_size2 = max(1, horizontal.shape[1]//100)
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT,(v_size2,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    h_final = cv2.dilate(h_final, h_struct2, iterations = 1)

    # 테이블 선의 굵기(line_wit)을 h_fianl 중간에서 위 아래로 선분을 그어 그 중 가장 두꺼운 선으로 기록한다.
    h_final[h_final >= 200] = 255
    wit = 0
    wit_list = []
    for p in h_final[:, h_final.shape[1]//2]:
        if p == 255:
            wit += 1
        else: 
            wit_list.append(wit)
            wit = 0
    line_wit = max(wit_list)
    line_wit

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    #mask는 테이블의 수직 선분과 수평 선분을 합친 윤곽선으로, 테이블을 나타낸다.
    mask = h_final + v_final

    cv2.imshow('mask', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 테이블의 수평선, 수직선이 교차하는 부분만 표시하는 배열이다.
    joints = cv2.bitwise_and(v_final, h_final)

    cv2.imshow('joint', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 수평, 수직선이 교차하는 지점에서 여러 점들이 합쳐져 있는데, 이 각각 점뭉치에서 하나의 점씩만 남기고 다시 joints에 저장한다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] !=0:
                joints[i - 2: i + 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = {}
    one_line = []
    tmp = []
    k = 0
    
    # 위에서 만든 joints에서 정리한 교차점들의 좌표만 one_line이라는 배열에 차곡차곡 넣는다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] == 255:
                one_line.append((i,j))

    # 이 one_line은 모든 점이 순서 상관없이 섞여있는데, 이제 점들을 각각 라인에 따라서 순서를 정렬해 dictionary로 만든다.
    fst = one_line[0]
    for pos in one_line:
        plow = fst[0]
        allow = int(joints.shape[0] * 0.03)
        rng = range(max(0, plow - allow), min(joints.shape[0] - 1, plow + allow))
        if pos[0] in rng:
            tmp.append(pos)
        else:
            sorted(tmp, key=second)
            k += 1
            points['line' + str(k)] = tmp
            tmp = []
            fst = pos
            tmp.append(pos)
    k += 1
    points['line' + str(k)] = tmp

    # 점 주위에 이 점이 시작점인지 끝점인지, 보조점인지 확인하기위해 점 주위로 박스를 만들어주는 기능
    depth = max(3, v_size2 + line_wit)
    box = np.zeros_like(mask)
    for i in range(1, len(points) + 1):
        line = points['line' + str(i)]
        for idx, p in enumerate(line):
            box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
            box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
            box[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
            box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)] = 255

    boxed = box + mask
    cv2.imshow('boxed', boxed)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 좌표에 위에서 만든 박스에 상하좌우에 닿는 점이 있는 지를 기준으로 최대 4가지 조건을 부여해준다.
    mask[mask > 240] = 255
    depth = max(3, v_size2 + line_wit)
    c_points = copy.deepcopy(points)
    for i in range(1, len(c_points) + 1):
        line = c_points['line' + str(i)]
        for idx, p in enumerate(line):
            tmp = ''
            c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
            c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
            c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
            c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)]
            if c1:
                tmp += 'c1'
            if c2:
                tmp += 'c2'
            if c3:
                tmp += 'c3'
            if c4:
                tmp += 'c4'
            line[idx] = (p, tmp)

    # 좌표에 부여된 조건을 기준으로 점들을 총 4가지로 중복을 허락해서 분류한다. 
    # 예를들어 어떤 점은 한 테이블의 시작점이며 동시에 다른 테이블의 끝점이 되며, 그 옆 테이블의 보조점이 될 수 도 있다.
    # 박스의 크기 때문에 점들이 제대로 파악되지 않을 경우 depth를 늘려가며, 시행합니다.
    for depth_increase in range(2, 5):
        allow = int(joints.shape[1] * 0.02)
        is_st = []
        is_end = []
        is_right = []
        is_bot = []
        for i in range(1, len(c_points) + 1):
            line = c_points['line' + str(i)]
            for idx, p in enumerate(line):
                if 'c3c4' in p[1]: 
                    is_st.append(p[0])
                if 'c1c2' in p[1]:
                    is_end.append(p[0])
                if 'c2c3' in p[1]:
                    is_right.append(p[0])
                if 'c1' in p[1] and 'c4' in p[1]:
                    is_bot.append(p[0])
        if len(is_st) == len(is_right) == len(is_bot) == len(is_end):
            break
        if depth_increase == 5:
            print("depth를 라인 두께의 5배까지 늘려봤지만 효과가 없었습니다.")
        else:
            print("시작점과 다른 점들의 수가 맞지 않습니다.")
            print("depth를 더 늘려 수정진행합니다.")
            # 점 주위에 박스처리 해주는 기능
            depth = max(3, v_size2 + line_wit * depth_increase)
            box = np.zeros_like(mask)
            for i in range(1, len(points) + 1):
                line = points['line' + str(i)]
                for idx, p in enumerate(line):
                    box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
                    box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)] = 255
            boxed = box + mask
            cv2.imshow('boxed', boxed)
            cv2.waitKey(0)
            cv2.destroyAllWindows() 
            cv2.waitKey(1)

            # 좌표에 컨디션을 부과해주는 함수
            mask[mask > 240] = 255
            depth = max(3, v_size2 + line_wit * depth_increase)
            c_points = copy.deepcopy(points)
            for i in range(1, len(c_points) + 1):
                line = c_points['line' + str(i)]
                for idx, p in enumerate(line):
                    tmp = ''
                    c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
                    c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
                    c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
                    c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)]
                    if c1:
                        tmp += 'c1'
                    if c2:
                        tmp += 'c2'
                    if c3:
                        tmp += 'c3'
                    if c4:
                        tmp += 'c4'
                    line[idx] = (p, tmp)
        

    # 이제 위에서 선별한 점들을 기준으로 각 테이블의 시작점과 끝점 리스트를 만든다.
    st = []
    end = []
    allow = int(joints.shape[1] * 0.02)
    # 스타트 다음에 나오는 right 는 무조건 직전 스타트와 사각형을 이룬다.
    for idx, p in enumerate(is_st):
        st.append(p)
        rgt = is_right[idx]
        for thing in is_bot:
            if thing[1] in range(p[1] - allow, p[1] + allow ):
                bot = thing
                is_bot.remove(thing)
                break
        for thing in is_end:
            if thing[0] in range(bot[0] - allow, bot[0] + allow ) and thing[1] in range(rgt[1] - allow, rgt[1] + allow):
                end.append(thing)
                is_end.remove(thing)
                break
    # 이미지를 주어진 좌표에 따라 자르는 함수, make_crack으로 이미지를 나눠주고, 각각의 이미지에 대해 테서락트를 실행한다.
    img = Image.fromarray(gray)
    img_list = make_crack(ful_img=img, st=st, end=end, line_wit=line_wit)  
    result = []
    for idx, image in enumerate(img_list):
        if idx != 18:
            result.append(pytesseract.image_to_string(image))
        else:
            for data in sub_puzzle(image):
                result.append(data)
    return result

In [25]:
def make_puzzle(url):
    src = cv2.imread(url)
    if src.shape[2] ==3:
        gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    horizontal = bw.copy()
    vertical = bw.copy()
    h_size = horizontal.shape[1]//5
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    v_size = vertical.shape[0]//5
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    joints = cv2.bitwise_and(v_final, h_final)
    mask = v_final + h_final
    
    cv2.imshow('img', src)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    cv2.imshow('table', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    cv2.imshow('joints', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] != 0:
                joints[i - 2: i+ 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = []
    tmp = []
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if len(tmp) == 2:
                if tmp[0][1] > tmp[1][1]:
                    points.append(tmp[1])
                    points.append(tmp[0])
                else:
                    points.append(tmp[0])
                    points.append(tmp[1])
                tmp = []
            if joints[i][j] == 255:
                tmp.append((i,j ))
    points = np.array(points)


    mask_st = []
    mask_end = []
    for idx in range(len(points)):
        if idx % 2 == 0:
            mask_st.append(idx)
        else:
            mask_end.append(idx)
    mask_st = mask_st[:-1]
    mask_end = mask_end[1:]
    st = points[mask_st]
    end = points[mask_end]

    ful_img = Image.open(url)
    img_list = make_crack(ful_img, st, end)
    for img in img_list:
        print(pytesseract.image_to_string(img))
        print("\n")
    return None


In [1]:
def make_crack_box(ful_img, st, end, line_wit):
    img_list = []

    for i in range(len(st)):
        tmp = ful_img.crop((st[i][1]+ line_wit *2, st[i][0] + line_wit * 2, end[i][1], end[i][0]))
        img_list.append(tmp)
    return img_list

def sub_puzzle(img):
    gray = np.array(img)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)

    cv2.destroyAllWindows()
    cv2.waitKey(1)

    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    cv2.imshow('bw', bw)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    horizontal = bw.copy()
    vertical = bw.copy()



    v_size = vertical.shape[0]//2
    v_size2 = max(1, vertical.shape[0]//100)
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size2))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    v_final = cv2.dilate(v_final, v_struct2, iterations=1)
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    h_size = horizontal.shape[1]//30
    # h_size2 = max(1, horizontal.shape[1]//100)
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT,(v_size2,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    h_final = cv2.dilate(h_final, h_struct2, iterations = 1)
    h_size

    # 
    h_final[h_final >= 200] = 255
    wit = 0
    wit_list = []

    for i in range(h_final.shape[1]):
        if sum(h_final[:, i]) >= 255 * 2 :
            break
    for p in h_final[:,i]:
        if p == 255:
            wit += 1
        else: 
            wit_list.append(wit)
            wit = 0
    line_wit = max(wit_list)
    line_wit

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    mask = h_final + v_final

    cv2.imshow('mask', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    joints = cv2.bitwise_and(v_final, h_final)
    cv2.imshow('joint', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    def second(x):
        return x[1]

    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] !=0:
                joints[i - 2: i + 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = {}
    one_line = []
    tmp = []
    k = 0

    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] == 255:
                one_line.append((i,j))


    fst = one_line[0]
    for pos in one_line:
        plow = fst[0]
        allow = int(joints.shape[0] * 0.03)
        rng = range(max(0, plow - allow), min(joints.shape[0] - 1, plow + allow))
        if pos[0] in rng:
            tmp.append(pos)
        else:
            sorted(tmp, key=second)
            k += 1
            points['line' + str(k)] = tmp
            tmp = []
            fst = pos
            tmp.append(pos)
    k += 1
    points['line' + str(k)] = tmp


    # 점 주위에 박스처리 해주는 기능
    depth = max(3, v_size2 + line_wit * 2)
    box = np.zeros_like(mask)
    for i in range(1, len(points) + 1):
        line = points['line' + str(i)]
        for idx, p in enumerate(line):
            box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
            box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
            box[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
            box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)] = 255

    boxed = box + mask
    cv2.imshow('boxed', boxed)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    # 좌표에 컨디션을 부과해주는 함수
    mask[mask > 240] = 255
    depth = max(3, v_size2 + line_wit * 2)
    c_points = copy.deepcopy(points)
    for i in range(1, len(c_points) + 1):
        line = c_points['line' + str(i)]
        for idx, p in enumerate(line):
            tmp = ''
            c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
            c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
            c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
            c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)]
            if c1:
                tmp += 'c1'
            if c2:
                tmp += 'c2'
            if c3:
                tmp += 'c3'
            if c4:
                tmp += 'c4'
            line[idx] = (p, tmp)
    # 좌표를 각각 시작점, 보조점, 끝점으로 분류해주는 기능.
    for depth_increase in range(3, 6):
        allow = int(joints.shape[1] * 0.02)
        is_st = []
        is_end = []
        is_right = []
        is_bot = []
        for i in range(1, len(c_points) + 1):
            line = c_points['line' + str(i)]
            for idx, p in enumerate(line):
                if 'c3c4' in p[1]: 
                    is_st.append(p[0])
                if 'c1c2' in p[1]:
                    is_end.append(p[0])
                if 'c2c3' in p[1]:
                    is_right.append(p[0])
                if 'c1' in p[1] and 'c4' in p[1]:
                    is_bot.append(p[0])
        if len(is_st) == len(is_right) == len(is_bot) == len(is_end):
            break
        else:
            print("시작점과 다른 점들의 수가 맞지 않습니다.")
            print("depth를 더 늘려 수정진행합니다.")
            depth = max(3, v_size2 + line_wit * depth_increase)
            box = np.zeros_like(mask)
            for i in range(1, len(points) + 1):
                line = points['line' + str(i)]
                for idx, p in enumerate(line):
                    box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
                    box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)] = 255

            boxed = box + mask
            cv2.imshow('boxed', boxed)
            cv2.waitKey(0)
            cv2.destroyAllWindows() 
            cv2.waitKey(1)
            # 좌표에 컨디션을 부과해주는 함수
            mask[mask > 240] = 255
            depth = max(3, v_size2 + line_wit * depth_increase)
            c_points = copy.deepcopy(points)
            for i in range(1, len(c_points) + 1):
                line = c_points['line' + str(i)]
                for idx, p in enumerate(line):
                    tmp = ''
                    c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
                    c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
                    c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
                    c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)]
                    if c1:
                        tmp += 'c1'
                    if c2:
                        tmp += 'c2'
                    if c3:
                        tmp += 'c3'
                    if c4:
                        tmp += 'c4'
                    line[idx] = (p, tmp)


    # 자를 좌표를 정해주는 기능
    st = []
    end = []
    allow = int(joints.shape[1] * 0.02)
    # 스타트 다음에 나오는 right 는 무조건 직전 스타트와 사각형을 이룬다.
    for idx, p in enumerate(is_st):
        st.append(p)
        rgt = is_right[idx]
        for thing in is_bot:
            if thing[1] in range(p[1] - allow, p[1] + allow ):
                bot = thing
                is_bot.remove(thing)
                break
        for thing in is_end:
            if thing[0] in range(bot[0] - allow, bot[0] + allow ) and thing[1] in range(rgt[1] - allow, rgt[1] + allow):
                end.append(thing)
                is_end.remove(thing)
                break
    # 13번 박스처리를 위한 코드에서만 필요.
    st = sorted(st, key=second)
    end = sorted(end, key=second)
    img = Image.fromarray(gray)
    img_list = make_crack_box(ful_img=img, st=st, end=end, line_wit=line_wit)
    choice = ['Statutory Employee', 'Retirement Plan', 'Third-party sick pay']
    result = []
    for idx, image in enumerate(img_list):
        gr = np.array(image)
        ret, thresh = cv2.threshold(gr, 70 ,255, cv2.THRESH_BINARY_INV)
        print(np.sum(thresh)/thresh.size)
        if np.sum(thresh)/thresh.size > 5:
            result.append("{}: Yes".format(choice[idx]))
        else:
            result.append("{}: No".format(choice[idx]))
    return result

In [24]:
make_puzzle2(url=url)

a. Employee’s Social Security Number
OMB No. 1545-0008 This information is being furnished to the Internet Revenue Service. if you are required to ﬁle
a tex return, a negligence penalty or other sanction maybe imposed on you if this income is taxable and you fail to
report it.
b. Employer‘s Identification Number
(EIN)
35 — 9990000
d. Control Number
1 Wages, Tips and other
compensation
24267 . 78
2 Federal Income Tax withheld
2838.60
c. Employer‘s Name, Address, and ZIP Code
DFAS ATTN: YUNSOO/JUNG
8899 EAST 56TH STREET

INDIANAPOLIS IN 46249—2410
3 Social Security Wages
2 3 6 6 7 . 7 8
4 Social Security Tax withheld
1 4 6 7 . 4 0
5 Medicare Wages and Tips
23667 . 78
6 Medicare Tax withheld
3 4 3 . 1 8
7 Social Security Tips
8 Allocated Tips
e. Employee’s Name, Address, and ZIP Code
Jonadan

GEWHQATO 6411

10 Dependent Care Benefits
12 See Instructions for box 12

AA16 387.08
14 See Instructions for box 14
13
15 State
VA
Employer’s State ID Number
3035999000F001
16 State Wages,
Tips and 

In [59]:
url2 = '/Users/yunsu/desktop/python_ocr/Target_OCR.png'


In [60]:
print(pytesseract.image_to_string(Image.open(url2)))


, 2 no .u — -" Socia‘ Security Number

‘ OMB No. 1545-0908 This information is being furnished to the internal Revenue Service. if you are F%mred L 3764:“~
tax return, a negligence penalty or other sanction may be imposed on you if this income is taxabie and 3401,. {an .0 raw .,

identiﬁcation Number (E'N) d. Control Number 1 Wages, Tips, and other compensation Federal income ,ax withi'ieic

5 24267.78 2838.60

ame. Address, and Z'P Code 3 Social Security Wages 4 Social Security Tax withheld
23667 78 1467.40

:3“ __ A‘I‘T'N: DPASIN/JAREA .
a a;— :— 2.1.3: 56TH STREET . . . .
3“,: 2H".PCL:S IN 45249 —2410 5 Medicare Wages and Tips 6 Medimre Tax Withheld
23667.78 343.18
7 Social Security tips 8 Allocated ips

.7, ~ , " ‘ " ‘ ' 10 Dependent Care Beneﬁts

12 See instructions for box 14 See instructions for box 14

 

S

./

     
  
        
 

fa. W’s Name. Address, and ZiP Code
i m2”: 641‘.

       
     
  

1 3 Statuto Retirement Third-party
D Empioyrge Plan D sick pay

% Wages, Tips, 

In [56]:
print(pytesseract.image_to_string(Image.open(url)))


1 Wages. Tips. and other compensation 1
24267.78

3 Social Security Wages
23667.78

5 Medicare Wages and Tips
23667.78
7 Social Security tips

12 See instructions for box .124


In [8]:
make_puzzle(url)


1 Wages. Tips, and other compensation
24267.78


3 Social Security Wages
23667.78


5 Medicare Wages and Tips
23667.78


'7 Social Security tips





12 See instructions for box 12

AA16 387.08




In [6]:
def second(x):
    return x[1]

def make_crack(ful_img, st, end, line_wit):
    img_list = []

    for i in range(len(st)):
        tmp = ful_img.crop((st[i][1]+ line_wit, st[i][0] + line_wit, end[i][1], end[i][0]))
        img_list.append(tmp)
    return img_list


def make_puzzle2(url):
    # 이미지 읽어오기
    src = cv2.imread(url)
    cv2.imshow('src', src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    #이미지 그레이스케일로 변환.
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    # 그레이스케일 이미지를 흑백 반전시키고, Adaptive thresh mean 을 적용해서 잡음 제거해준다.
    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    cv2.imshow('bw', bw)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 수직, 수평 직선을 확인하는 기능. 코드 리팩토링 필요.
    horizontal = bw.copy()
    vertical = bw.copy()
    v_size = vertical.shape[0]//20
    v_size2 = max(1, vertical.shape[0]//100)
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size2))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    v_final = cv2.dilate(v_final, v_struct2, iterations=1)
    
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # h_final은 이미지에서 테이블 수직 선만 표시하는 배열이다.
    h_size = horizontal.shape[1]//20
    # h_size2 = max(1, horizontal.shape[1]//100)
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT,(v_size2,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    h_final = cv2.dilate(h_final, h_struct2, iterations = 1)

    # 테이블 선의 굵기(line_wit)을 h_fianl 중간에서 위 아래로 선분을 그어 그 중 가장 두꺼운 선으로 기록한다.
    h_final[h_final >= 200] = 255
    wit = 0
    wit_list = []
    for p in h_final[:, h_final.shape[1]//2]:
        if p == 255:
            wit += 1
        else: 
            wit_list.append(wit)
            wit = 0
    line_wit = max(wit_list)
    line_wit

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    #mask는 테이블의 수직 선분과 수평 선분을 합친 윤곽선으로, 테이블을 나타낸다.
    mask = h_final + v_final

    cv2.imshow('mask', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 테이블의 수평선, 수직선이 교차하는 부분만 표시하는 배열이다.
    joints = cv2.bitwise_and(v_final, h_final)

    cv2.imshow('joint', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 수평, 수직선이 교차하는 지점에서 여러 점들이 합쳐져 있는데, 이 각각 점뭉치에서 하나의 점씩만 남기고 다시 joints에 저장한다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] !=0:
                joints[i - 2: i + 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = {}
    one_line = []
    tmp = []
    k = 0
    
    # 위에서 만든 joints에서 정리한 교차점들의 좌표만 one_line이라는 배열에 차곡차곡 넣는다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] == 255:
                one_line.append((i,j))

    # 이 one_line은 모든 점이 순서 상관없이 섞여있는데, 이제 점들을 각각 라인에 따라서 순서를 정렬해 dictionary로 만든다.
    fst = one_line[0]
    for pos in one_line:
        plow = fst[0]
        allow = int(joints.shape[0] * 0.03)
        rng = range(max(0, plow - allow), min(joints.shape[0] - 1, plow + allow))
        if pos[0] in rng:
            tmp.append(pos)
        else:
            sorted(tmp, key=second)
            k += 1
            points['line' + str(k)] = tmp
            tmp = []
            fst = pos
            tmp.append(pos)
    k += 1
    points['line' + str(k)] = tmp

    # 점 주위에 이 점이 시작점인지 끝점인지, 보조점인지 확인하기위해 점 주위로 박스를 만들어주는 기능
    depth = max(3, v_size2 + line_wit)
    box = np.zeros_like(mask)
    for i in range(1, len(points) + 1):
        line = points['line' + str(i)]
        for idx, p in enumerate(line):
            box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
            box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
            box[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
            box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)] = 255

    boxed = box + mask
    cv2.imshow('boxed', boxed)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 좌표에 위에서 만든 박스에 상하좌우에 닿는 점이 있는 지를 기준으로 최대 4가지 조건을 부여해준다.
    mask[mask > 240] = 255
    depth = max(3, v_size2 + line_wit)
    c_points = copy.deepcopy(points)
    for i in range(1, len(c_points) + 1):
        line = c_points['line' + str(i)]
        for idx, p in enumerate(line):
            tmp = ''
            c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
            c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
            c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
            c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)]
            if c1:
                tmp += 'c1'
            if c2:
                tmp += 'c2'
            if c3:
                tmp += 'c3'
            if c4:
                tmp += 'c4'
            line[idx] = (p, tmp)

    # 좌표에 부여된 조건을 기준으로 점들을 총 4가지로 중복을 허락해서 분류한다. 
    # 예를들어 어떤 점은 한 테이블의 시작점이며 동시에 다른 테이블의 끝점이 되며, 그 옆 테이블의 보조점이 될 수 도 있다.
    # 박스의 크기 때문에 점들이 제대로 파악되지 않을 경우 depth를 늘려가며, 시행합니다.
    for depth_increase in range(2, 5):
        allow = int(joints.shape[1] * 0.02)
        is_st = []
        is_end = []
        is_right = []
        is_bot = []
        for i in range(1, len(c_points) + 1):
            line = c_points['line' + str(i)]
            for idx, p in enumerate(line):
                if 'c3c4' in p[1]: 
                    is_st.append(p[0])
                if 'c1c2' in p[1]:
                    is_end.append(p[0])
                if 'c2c3' in p[1]:
                    is_right.append(p[0])
                if 'c1' in p[1] and 'c4' in p[1]:
                    is_bot.append(p[0])
        if len(is_st) == len(is_right) == len(is_bot) == len(is_end):
            break
        if depth_increase == 5:
            print("depth를 라인 두께의 5배까지 늘려봤지만 효과가 없었습니다.")
        else:
            print("시작점과 다른 점들의 수가 맞지 않습니다.")
            print("depth를 더 늘려 수정진행합니다.")
            # 점 주위에 박스처리 해주는 기능
            depth = max(3, v_size2 + line_wit * depth_increase)
            box = np.zeros_like(mask)
            for i in range(1, len(points) + 1):
                line = points['line' + str(i)]
                for idx, p in enumerate(line):
                    box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
                    box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)] = 255
            boxed = box + mask
            cv2.imshow('boxed', boxed)
            cv2.waitKey(0)
            cv2.destroyAllWindows() 
            cv2.waitKey(1)

            # 좌표에 컨디션을 부과해주는 함수
            mask[mask > 240] = 255
            depth = max(3, v_size2 + line_wit * depth_increase)
            c_points = copy.deepcopy(points)
            for i in range(1, len(c_points) + 1):
                line = c_points['line' + str(i)]
                for idx, p in enumerate(line):
                    tmp = ''
                    c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
                    c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
                    c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
                    c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)]
                    if c1:
                        tmp += 'c1'
                    if c2:
                        tmp += 'c2'
                    if c3:
                        tmp += 'c3'
                    if c4:
                        tmp += 'c4'
                    line[idx] = (p, tmp)
        

    # 이제 위에서 선별한 점들을 기준으로 각 테이블의 시작점과 끝점 리스트를 만든다.
    st = []
    end = []
    allow = int(joints.shape[1] * 0.02)
    # 스타트 다음에 나오는 right 는 무조건 직전 스타트와 사각형을 이룬다.
    for idx, p in enumerate(is_st):
        st.append(p)
        rgt = is_right[idx]
        for thing in is_bot:
            if thing[1] in range(p[1] - allow, p[1] + allow ):
                bot = thing
                is_bot.remove(thing)
                break
        for thing in is_end:
            if thing[0] in range(bot[0] - allow, bot[0] + allow ) and thing[1] in range(rgt[1] - allow, rgt[1] + allow):
                end.append(thing)
                is_end.remove(thing)
                break
    # 이미지를 주어진 좌표에 따라 자르는 함수, make_crack으로 이미지를 나눠주고, 각각의 이미지에 대해 테서락트를 실행한다.
    img = Image.fromarray(gray)
    img_list = make_crack(ful_img=img, st=st, end=end)   
    for image in img_list:
        print(pytesseract.image_to_string(image))
    return None

In [ ]:
def make_puzzle2(url):
    # 이미지 읽어오기
    src = cv2.imread(url)
    cv2.imshow('src', src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    #이미지 그레이스케일로 변환.
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    # 그레이스케일 이미지를 흑백 반전시키고, Adaptive thresh mean 을 적용해서 잡음 제거해준다.
    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    cv2.imshow('bw', bw)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 수직, 수평 직선을 확인하는 기능. 코드 리팩토링 필요.
    horizontal = bw.copy()
    vertical = bw.copy()
    v_size = vertical.shape[0]//20
    v_size2 = max(1, vertical.shape[0]//100)
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size2))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    v_final = cv2.dilate(v_final, v_struct2, iterations=1)
    
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # h_final은 이미지에서 테이블 수직 선만 표시하는 배열이다.
    h_size = horizontal.shape[1]//20
    # h_size2 = max(1, horizontal.shape[1]//100)
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT,(v_size2,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    h_final = cv2.dilate(h_final, h_struct2, iterations = 1)

    # 테이블 선의 굵기(line_wit)을 h_fianl 중간에서 위 아래로 선분을 그어 그 중 가장 두꺼운 선으로 기록한다.
    h_final[h_final >= 200] = 255
    wit = 0
    wit_list = []
    for p in h_final[:, h_final.shape[1]//2]:
        if p == 255:
            wit += 1
        else: 
            wit_list.append(wit)
            wit = 0
    line_wit = max(wit_list)
    line_wit

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    #mask는 테이블의 수직 선분과 수평 선분을 합친 윤곽선으로, 테이블을 나타낸다.
    mask = h_final + v_final

    cv2.imshow('mask', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 테이블의 수평선, 수직선이 교차하는 부분만 표시하는 배열이다.
    joints = cv2.bitwise_and(v_final, h_final)

    cv2.imshow('joint', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # joints는 수평, 수직선이 교차하는 지점에서 여러 점들이 합쳐져 있는데, 이 각각 점뭉치에서 하나의 점씩만 남기고 다시 joints에 저장한다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] !=0:
                joints[i - 2: i + 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = {}
    one_line = []
    tmp = []
    k = 0
    
    # 위에서 만든 joints에서 정리한 교차점들의 좌표만 one_line이라는 배열에 차곡차곡 넣는다.
    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] == 255:
                one_line.append((i,j))

    # 이 one_line은 모든 점이 순서 상관없이 섞여있는데, 이제 점들을 각각 라인에 따라서 순서를 정렬해 dictionary로 만든다.
    fst = one_line[0]
    for pos in one_line:
        plow = fst[0]
        allow = int(joints.shape[0] * 0.03)
        rng = range(max(0, plow - allow), min(joints.shape[0] - 1, plow + allow))
        if pos[0] in rng:
            tmp.append(pos)
        else:
            sorted(tmp, key=second)
            k += 1
            points['line' + str(k)] = tmp
            tmp = []
            fst = pos
            tmp.append(pos)
    k += 1
    points['line' + str(k)] = tmp

    # 점 주위에 이 점이 시작점인지 끝점인지, 보조점인지 확인하기위해 점 주위로 박스를 만들어주는 기능
    depth = max(3, v_size2 + line_wit)
    box = np.zeros_like(mask)
    for i in range(1, len(points) + 1):
        line = points['line' + str(i)]
        for idx, p in enumerate(line):
            box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
            box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
            box[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
            box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)] = 255

    boxed = box + mask
    cv2.imshow('boxed', boxed)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    
    # 좌표에 위에서 만든 박스에 상하좌우에 닿는 점이 있는 지를 기준으로 최대 4가지 조건을 부여해준다.
    mask[mask > 240] = 255
    depth = max(3, v_size2 + line_wit)
    c_points = copy.deepcopy(points)
    for i in range(1, len(c_points) + 1):
        line = c_points['line' + str(i)]
        for idx, p in enumerate(line):
            tmp = ''
            c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
            c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
            c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
            c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)]
            if c1:
                tmp += 'c1'
            if c2:
                tmp += 'c2'
            if c3:
                tmp += 'c3'
            if c4:
                tmp += 'c4'
            line[idx] = (p, tmp)

    # 좌표에 부여된 조건을 기준으로 점들을 총 4가지로 중복을 허락해서 분류한다. 
    # 예를들어 어떤 점은 한 테이블의 시작점이며 동시에 다른 테이블의 끝점이 되며, 그 옆 테이블의 보조점이 될 수 도 있다.
    # 박스의 크기 때문에 점들이 제대로 파악되지 않을 경우 depth를 늘려가며, 시행합니다.
    for depth_increase in range(2, 5):
        allow = int(joints.shape[1] * 0.02)
        is_st = []
        is_end = []
        is_right = []
        is_bot = []
        for i in range(1, len(c_points) + 1):
            line = c_points['line' + str(i)]
            for idx, p in enumerate(line):
                if 'c3c4' in p[1]: 
                    is_st.append(p[0])
                if 'c1c2' in p[1]:
                    is_end.append(p[0])
                if 'c2c3' in p[1]:
                    is_right.append(p[0])
                if 'c1' in p[1] and 'c4' in p[1]:
                    is_bot.append(p[0])
        if len(is_st) == len(is_right) == len(is_bot) == len(is_end):
            break
        if depth_increase == 5:
            print("depth를 라인 두께의 5배까지 늘려봤지만 효과가 없었습니다.")
        else:
            print("시작점과 다른 점들의 수가 맞지 않습니다.")
            print("depth를 더 늘려 수정진행합니다.")
            # 점 주위에 박스처리 해주는 기능
            depth = max(3, v_size2 + line_wit * depth_increase)
            box = np.zeros_like(mask)
            for i in range(1, len(points) + 1):
                line = points['line' + str(i)]
                for idx, p in enumerate(line):
                    box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
                    box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)] = 255
            boxed = box + mask
            cv2.imshow('boxed', boxed)
            cv2.waitKey(0)
            cv2.destroyAllWindows() 
            cv2.waitKey(1)

            # 좌표에 컨디션을 부과해주는 함수
            mask[mask > 240] = 255
            depth = max(3, v_size2 + line_wit * depth_increase)
            c_points = copy.deepcopy(points)
            for i in range(1, len(c_points) + 1):
                line = c_points['line' + str(i)]
                for idx, p in enumerate(line):
                    tmp = ''
                    c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
                    c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
                    c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
                    c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)]
                    if c1:
                        tmp += 'c1'
                    if c2:
                        tmp += 'c2'
                    if c3:
                        tmp += 'c3'
                    if c4:
                        tmp += 'c4'
                    line[idx] = (p, tmp)
        

    # 이제 위에서 선별한 점들을 기준으로 각 테이블의 시작점과 끝점 리스트를 만든다.
    st = []
    end = []
    allow = int(joints.shape[1] * 0.02)
    # 스타트 다음에 나오는 right 는 무조건 직전 스타트와 사각형을 이룬다.
    for idx, p in enumerate(is_st):
        st.append(p)
        rgt = is_right[idx]
        for thing in is_bot:
            if thing[1] in range(p[1] - allow, p[1] + allow ):
                bot = thing
                is_bot.remove(thing)
                break
        for thing in is_end:
            if thing[0] in range(bot[0] - allow, bot[0] + allow ) and thing[1] in range(rgt[1] - allow, rgt[1] + allow):
                end.append(thing)
                is_end.remove(thing)
                break
    # 이미지를 주어진 좌표에 따라 자르는 함수, make_crack으로 이미지를 나눠주고, 각각의 이미지에 대해 테서락트를 실행한다.
    img = Image.fromarray(gray)
    img_list = make_crack(ful_img=img, st=st, end=end, line_wit=line_wit)  
    result = []
    for idx, image in enumerate(img_list):
        if idx != 18:
            result.append(pytesseract.image_to_string(image))
        else:
            for data in sub_puzzle(image):
                result.append(data)
    return result

In [ ]:
def make_crack_box(ful_img, st, end, line_wit):
    img_list = []

    for i in range(len(st)):
        tmp = ful_img.crop((st[i][1]+ line_wit *2, st[i][0] + line_wit * 2, end[i][1], end[i][0]))
        img_list.append(tmp)
    return img_list

def sub_puzzle(img):
    gray = np.array(img)
    cv2.imshow('gray', gray)
    cv2.waitKey(0)

    cv2.destroyAllWindows()
    cv2.waitKey(1)

    bw = cv2.adaptiveThreshold(~gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, -5)
    cv2.imshow('bw', bw)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    horizontal = bw.copy()
    vertical = bw.copy()



    v_size = vertical.shape[0]//2
    v_size2 = max(1, vertical.shape[0]//100)
    v_struct = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size))
    v_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, v_size2))
    v_sub = cv2.erode(vertical, v_struct, iterations=1)
    v_final = cv2.dilate(v_sub, v_struct, iterations=1)
    v_final = cv2.dilate(v_final, v_struct2, iterations=1)
    cv2.imshow('vertical', v_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    h_size = horizontal.shape[1]//30
    # h_size2 = max(1, horizontal.shape[1]//100)
    h_struct = cv2.getStructuringElement(cv2.MORPH_RECT,(h_size,1))
    h_struct2 = cv2.getStructuringElement(cv2.MORPH_RECT,(v_size2,1))
    h_sub = cv2.erode(horizontal, h_struct, iterations = 1)
    h_final = cv2.dilate(h_sub, h_struct, iterations = 1)
    h_final = cv2.dilate(h_final, h_struct2, iterations = 1)
    h_size

    # 
    h_final[h_final >= 200] = 255
    wit = 0
    wit_list = []

    for i in range(h_final.shape[1]):
        if sum(h_final[:, i]) >= 255 * 2 :
            break
    for p in h_final[:,i]:
        if p == 255:
            wit += 1
        else: 
            wit_list.append(wit)
            wit = 0
    line_wit = max(wit_list)
    line_wit

    cv2.imshow('horizontal', h_final)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)
    mask = h_final + v_final

    cv2.imshow('mask', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    joints = cv2.bitwise_and(v_final, h_final)
    cv2.imshow('joint', joints)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    def second(x):
        return x[1]

    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] !=0:
                joints[i - 2: i + 7, j - 2: j + 7] = 0
                joints[i][j] = 255
    points = {}
    one_line = []
    tmp = []
    k = 0

    for i in range(joints.shape[0]):
        for j in range(joints.shape[1]):
            if joints[i][j] == 255:
                one_line.append((i,j))


    fst = one_line[0]
    for pos in one_line:
        plow = fst[0]
        allow = int(joints.shape[0] * 0.03)
        rng = range(max(0, plow - allow), min(joints.shape[0] - 1, plow + allow))
        if pos[0] in rng:
            tmp.append(pos)
        else:
            sorted(tmp, key=second)
            k += 1
            points['line' + str(k)] = tmp
            tmp = []
            fst = pos
            tmp.append(pos)
    k += 1
    points['line' + str(k)] = tmp


    # 점 주위에 박스처리 해주는 기능
    depth = max(3, v_size2 + line_wit * 2)
    box = np.zeros_like(mask)
    for i in range(1, len(points) + 1):
        line = points['line' + str(i)]
        for idx, p in enumerate(line):
            box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
            box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
            box[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
            box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)] = 255

    boxed = box + mask
    cv2.imshow('boxed', boxed)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

    # 좌표에 컨디션을 부과해주는 함수
    mask[mask > 240] = 255
    depth = max(3, v_size2 + line_wit * 2)
    c_points = copy.deepcopy(points)
    for i in range(1, len(c_points) + 1):
        line = c_points['line' + str(i)]
        for idx, p in enumerate(line):
            tmp = ''
            c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
            c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
            c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                             max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
            c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                             min(joints.shape[1] -1 ,p[1] + depth)]
            if c1:
                tmp += 'c1'
            if c2:
                tmp += 'c2'
            if c3:
                tmp += 'c3'
            if c4:
                tmp += 'c4'
            line[idx] = (p, tmp)
    # 좌표를 각각 시작점, 보조점, 끝점으로 분류해주는 기능.
    for depth_increase in range(3, 6):
        allow = int(joints.shape[1] * 0.02)
        is_st = []
        is_end = []
        is_right = []
        is_bot = []
        for i in range(1, len(c_points) + 1):
            line = c_points['line' + str(i)]
            for idx, p in enumerate(line):
                if 'c3c4' in p[1]: 
                    is_st.append(p[0])
                if 'c1c2' in p[1]:
                    is_end.append(p[0])
                if 'c2c3' in p[1]:
                    is_right.append(p[0])
                if 'c1' in p[1] and 'c4' in p[1]:
                    is_bot.append(p[0])
        if len(is_st) == len(is_right) == len(is_bot) == len(is_end):
            break
        else:
            print("시작점과 다른 점들의 수가 맞지 않습니다.")
            print("depth를 더 늘려 수정진행합니다.")
            depth = max(3, v_size2 + line_wit * depth_increase)
            box = np.zeros_like(mask)
            for i in range(1, len(points) + 1):
                line = points['line' + str(i)]
                for idx, p in enumerate(line):
                    box[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)] = 255
                    box[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] = 255
                    box[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)] = 255

            boxed = box + mask
            cv2.imshow('boxed', boxed)
            cv2.waitKey(0)
            cv2.destroyAllWindows() 
            cv2.waitKey(1)
            # 좌표에 컨디션을 부과해주는 함수
            mask[mask > 240] = 255
            depth = max(3, v_size2 + line_wit * depth_increase)
            c_points = copy.deepcopy(points)
            for i in range(1, len(c_points) + 1):
                line = c_points['line' + str(i)]
                for idx, p in enumerate(line):
                    tmp = ''
                    c1 = 255 in mask[max(0, p[0] - depth) , max(0, p[1] - depth) : min(joints.shape[1] - 1, p[1] + depth)]
                    c2 = 255 in mask[max(0, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), max(0, p[1] - depth)]
                    c3 = 255 in mask[min(joints.shape[0] - 1, p[0] + depth), 
                                     max(0, p[1] - depth) : min(joints.shape[1] -1 ,p[1] + depth)] 
                    c4 = 255 in mask[min(joints.shape[0] - 1, p[0] - depth) : min(joints.shape[0] - 1, p[0] + depth), 
                                     min(joints.shape[1] -1 ,p[1] + depth)]
                    if c1:
                        tmp += 'c1'
                    if c2:
                        tmp += 'c2'
                    if c3:
                        tmp += 'c3'
                    if c4:
                        tmp += 'c4'
                    line[idx] = (p, tmp)


    # 자를 좌표를 정해주는 기능
    st = []
    end = []
    allow = int(joints.shape[1] * 0.02)
    # 스타트 다음에 나오는 right 는 무조건 직전 스타트와 사각형을 이룬다.
    for idx, p in enumerate(is_st):
        st.append(p)
        rgt = is_right[idx]
        for thing in is_bot:
            if thing[1] in range(p[1] - allow, p[1] + allow ):
                bot = thing
                is_bot.remove(thing)
                break
        for thing in is_end:
            if thing[0] in range(bot[0] - allow, bot[0] + allow ) and thing[1] in range(rgt[1] - allow, rgt[1] + allow):
                end.append(thing)
                is_end.remove(thing)
                break
    # 13번 박스처리를 위한 코드에서만 필요.
    st = sorted(st, key=second)
    end = sorted(end, key=second)
    img = Image.fromarray(gray)
    img_list = make_crack_box(ful_img=img, st=st, end=end, line_wit=line_wit)
    choice = ['Statutory Employee', 'Retirement Plan', 'Third-party sick pay']
    result = []
    for idx, image in enumerate(img_list):
        gr = np.array(image)
        ret, thresh = cv2.threshold(gr, 70 ,255, cv2.THRESH_BINARY_INV)
        print(np.sum(thresh)/thresh.size)
        if np.sum(thresh)/thresh.size > 5:
            result.append("{}: Yes".format(choice[idx]))
        else:
            result.append("{}: No".format(choice[idx]))
    return result